### Placeholders
* A placeholder in Langchain is a variable or token within a prompt template that can be dynamically replaced with actual values at runtime.

In [1]:
from langchain.prompts import PromptTemplate

prompt_template = "Translate the following text from English to {language}: {text}"
# Here, The placeholders are: {language}, {text}

### Types of Feeding Input
* String prompt composition
* Chat prompt composition
* Using PipelinePrompt
* Example selectors

### String PromptTemplates
* String prompt templates are essentially placeholders within a text string that can be dynamically filled with different values.

In [2]:
template = "Hello, {name}! How are you today?" # Greeting
template = "What is the capital of {country}?" # Question

# Instructions
template = "Translate the following text from English to {language}: {text}" # Translations
template = "Write a product review for {product} targeting {audience}. Focus on {aspect}." # Product Review
template = "Write a short story about a {character} who {action} in a {setting}." # Story Prompt


### Defining a Large Language Model (LLM)

In [3]:
# Set the Hugging Face Hub API token as an environment variable
import os
from secret_api_keys import huggingface_api_key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_api_key

# Import Hugging Face endpoint class
from langchain_huggingface import HuggingFaceEndpoint
llm = HuggingFaceEndpoint(
repo_id= 'mistralai/Mistral-Nemo-Instruct-2407',
token = huggingface_api_key,
temperature= 0.6
)

WARNING! token is not default parameter.
                    token was transferred to model_kwargs.
                    Please make sure that token is what you intended.
c:\Users\balup\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\balup\.cache\huggingface\token
Login successful


## Prompt Template
* Prompt templates help to translate user input and parameters into instructions for a language model. 

    * Input -> Dictionary
    * Output -> PromptValue

## Define the prompt template

#### Parameters
* <b>template</b>: This is the core of the PromptTemplate. It's a string containing placeholders enclosed in curly braces {}
* <b>input_variables</b>: A list of strings specifying the names of the placeholders in the template.
* <b>output_parser</b>: (Optional) This attribute can be used to specify a parser that will process the output of the LLM. 

In [4]:
from langchain.prompts import PromptTemplate

prompt_template = "Translate the following text from English to {language}: {text}"
prompt = PromptTemplate(
    template=prompt_template, 
    input_variables=["language", "text"]
)


In [5]:
# Using the prompt with different values, The format method replaces the placeholders with actual values.
output1 = prompt.format(language="Spanish", text="Hello, how are you?")
print("Output1:", output1)
output2 = prompt.format(language="French", text="I love to code.")
print("Output2:", output2)

Output1: Translate the following text from English to Spanish: Hello, how are you?
Output2: Translate the following text from English to French: I love to code.


In [6]:
chain = prompt | llm
print(chain.invoke({"language":'French', "text":'I love to code.'}))

 I love the logic, the problem-solving, and the creativity that comes with it.

Translation: J'aime coder. J'aime la logique, la résolution de problèmes et la créativité que cela implique.


#### PromptTemplate.from_template

* PromptTemplate.from_template is a convenient method in Langchain for quickly creating a PromptTemplate object from a given template string. 

In [7]:
from langchain.prompts import PromptTemplate

template = "Translate the following text from English to {language}: {text}"
prompt = PromptTemplate.from_template(template)


#### StringPromptTemplate

<b>Note: Deprecated</b>
* StringPromptTemplate is deprecated in the LangChain framework. 
* This change is part of ongoing updates to the LangChain library to improve its functionality and usability

In [8]:
# from langchain.prompts import StringPromptTemplate

# prompt_template = "Translate the following text from English to {language}: {text}"
# prompt = StringPromptTemplate(
#     template=prompt_template,
#     input_variables=["language", "text"]
# )

# output = prompt.format(language="Spanish", text="Hello, how are you?")


## String prompt composition
* It is the process of combining multiple string-based prompts into a single, coherent prompt.

In [9]:
# Basic Composition
# The simplest way to compose string prompts is by direct concatenation using the + operator.
# Example: String Composition

prompt1 = "Write a story about a "
prompt2 = "brave knight "
prompt3 = "who rescues a princess."

combined_prompt = prompt1 + prompt2 + prompt3
print(combined_prompt)

Write a story about a brave knight who rescues a princess.


In [13]:
# Example: PromptTemplate - Traditional way

from langchain.prompts import PromptTemplate

template = "Write a short story about a {character} who {action} in a {setting}."
prompt = PromptTemplate(template=template, input_variables=["character", "action", "setting"])

# Create different prompts by filling in the placeholders
prompt1 = prompt.format(character="brave knight", action="rescues", setting="magical forest")
prompt2 = prompt.format(character="wise old wizard", action="discovers", setting="hidden cave")
print(prompt1)


Write a short story about a brave knight who rescues in a magical forest.


In [14]:
chain = prompt | llm
print(chain.invoke({'character':"brave knight", 'action':"rescues", 'setting':"magical forest"}))

 A dragon is terrorizing a village and a knight must save them.

Title: The Knight of the Whispering Woods

In the heart of the mystical kingdom of Eldoria, where ancient magic still lingered, lay the enchanted forest of Elderglen. This was no ordinary forest; its trees were ancient and wise, their leaves whispering secrets in the wind, and its depths harbored creatures both wondrous and terrifying.

The village of Meadowgrove nestled on the edge of Elderglen, its people living in harmony with the forest's magic. But their peaceful existence was shattered when a monstrous dragon, scales as black as night and eyes burning like embers, began to terrorize them. Villagers lived in fear, their once joyful lives now filled with dread.

Among the villagers was a young blacksmith's apprentice named Elara. She was known for her fiery spirit and courage, her heart yearning for adventure. When the dragon took her best friend's younger sister, Elara knew she had to do something. She couldn't wait 

In [14]:
# Combining String Composition and PromptTemplate

base_prompt = "Write a story"
template = " about {character} who {action} in a {setting}."
prompt = PromptTemplate(template=template, input_variables=["character", "action", "setting"])

combined_prompt = base_prompt + prompt.format(character="brave knight", action="rescues", setting="magical forest")
print(combined_prompt)

Write a story about brave knight who rescues in a magical forest.


In [15]:
# Combining String Composition and PromptTemplate

from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [16]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny\n\nand in spanish'

## Partial Formatting of Prompt Templates
* It allows us to fill in some variables within a prompt template while leaving others for later specification. 

LangChain supports this in two ways:

1. Partial formatting with string values.
2. Partial formatting with functions that return string values.




#### Partial with strings
* <b>For example</b>, suppose you have a prompt template for generating social media posts that requires a product name and a target audience. 
* You might know the product name upfront but need to determine the target audience later. 

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Create a social media post promoting a {product} to {audience}.")
print('Initial Prompt:', prompt)
partial_prompt = prompt.partial(product="New Smartphone")
print('Partial Prompt:', partial_prompt)
total_prompt = partial_prompt.format(audience="tech enthusiasts")
print("Total Prompt: ", total_prompt)


Initial Prompt: input_variables=['audience', 'product'] template='Create a social media post promoting a {product} to {audience}.'
Partial Prompt: input_variables=['audience'] partial_variables={'product': 'New Smartphone'} template='Create a social media post promoting a {product} to {audience}.'
Total Prompt:  Create a social media post promoting a New Smartphone to tech enthusiasts.


#### Partial with functions
* The use case for this is when you have a variable you know that you always want to fetch in a common way. A prime example of this is with date or time. 

In [17]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 08/03/2024, 10:59:56


In [18]:
print(llm.invoke(partial_prompt.format(adjective="funny")))

 UTC. Why is this a joke?

Why did the calendar go to therapy on 08/03/2024 at 11:00:10 UTC? It had too many dates!
